In [1]:
!git pull origin master

From https://github.com/Sjsingh101/devanagari-to-roman-script-transliteration
 * branch            master     -> FETCH_HEAD
Already up to date.


In [64]:
import os
import re
import csv
import json
from datetime import datetime

In [11]:
BASE_PATH = os.path.abspath('')
path_input = os.path.join(BASE_PATH, "data")
path_hindi = os.path.join(BASE_PATH, "hindi")
path_english = os.path.join(BASE_PATH, "english")
files = os.listdir(path_input)
print(files)

['Hindi_OM.txt']


## Data Cleaning and Loading

In [42]:
for file_name in files:
    with open(os.path.join(path_input,file_name)) as file_read:
        data = file_read.read()
        data = re.sub('[0-9]',' ', data)
        data_list=data.replace('\n',' ').replace('/',' ').replace('.',' ').replace('(',' ').replace(')',' ').replace('?',' ').replace('-',' ').replace(',',' ').replace('.',' ').replace(':',' ').split(' ') 
        data_list = [ word for word in data_list if len(word)<20 ]
        data_list = list(filter(None, data_list))    
    with open(os.path.join(path_hindi,file_name), 'w') as hindi_file:
        hindi_file.writelines("%s\n" % word for word in set(data_list))


In [17]:
reader = csv.reader(open(os.path.join(BASE_PATH, "svar.csv"), 'r'))
vowels = {}
for row in reader:
	k, v = row
	vowels[k] = v

reader = csv.reader(open(os.path.join(BASE_PATH, "vyanjan.csv"), 'r'))
consonants = {}
for row in reader:
	k, v = row
	consonants[k] = v

## Unique Words

In [43]:
len(set(data_list))

1671

## Translitration

In [48]:
str1 = ""
for x in set(data_list):
	for y in x.split():
		for i in range(len(y)):
			if (i+1<len(y) and y[i+1].strip()==' ़'.strip()):
				c = y[i]+y[i+1]
				p=2
			else:
				c = y[i]
				p=1
			if (c in vowels.keys()):
				str1 = str1 + vowels[c]
			elif (c in consonants.keys()):
				if(i+p<len(y) and y[i+p] in consonants.keys()):
					if ((c=='झ' and i!=0) or (i!=0 and i+p+1<len(y) and y[i+p+1] in vowels.keys())): 
						str1 = str1 + consonants[c]
					else:
						str1 = str1 + consonants[c]+'a'
				else:
					str1 = str1 + consonants[c]
			elif y[i] in ['\n','\t',' ','!',',','।','-',':','\\','_'] or c.isalnum():
				str1 = str1 + c.replace('।','.')
		str1 = str1 + " "
	str1 = str1 + "\n"

7.73 ms ± 85.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [59]:
with open(os.path.join(path_english,file_name), 'w') as eng_file:
    eng_file.write(str1.replace(' ','').strip())

In [60]:
word_dict = {}
for file_name in files:
    hindi = [line.strip('\n').strip(')').strip('(').strip('।') for line in open(os.path.join(path_hindi,file_name), 'r')]    
    english = [line.strip('\n') for line in open(os.path.join(path_english,file_name), 'r')]
    word_dict.update(dict(zip(english, hindi))) 


In [65]:
timestamp = str(int(datetime.timestamp(datetime.now())))
with open(os.path.join(BASE_PATH,'data-'+timestamp+'.json'), 'w', encoding='utf-8') as final:
    json.dump(word_dict, final, ensure_ascii=False, indent=4)
